# Creation of fake dataset uisng "Faker" module for data analysis part

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.0 MB/s eta 0:00:00


In [3]:
import random
import json
from faker import Faker

fake = Faker()

# Generate fake teams
teams = [{"name": f"Team {chr(65+i)}", "country": fake.country()} for i in range(8)]

# Generate fake players
roles = ["Batsman", "Bowler", "All-rounder", "Wicketkeeper"]
players = [
    {
        "name": fake.name(),
        "age": random.randint(18, 35),
        "role": random.choice(roles),
        "team": random.choice(teams)["name"]
    }
    for _ in range(50)
]

# Generate fake matches
locations = [fake.city() for _ in range(10)]
matches = [
    {
        "match_id": i,
        "date": fake.date_between(start_date="-2y", end_date="today").strftime("%Y-%m-%d"),
        "location": random.choice(locations),
        "team1": random.choice(teams)["name"],
        "team2": random.choice(teams)["name"],
        "winner": random.choice([None, "team1", "team2"]),
    }
    for i in range(20)
]

# Export dataset as JSON
dataset = {"teams": teams, "players": players, "matches": matches}
with open("cricket_data.json", "w") as f:
    json.dump(dataset, f, indent=4)

print("Fake dataset created and saved as cricket_data.json")


Fake dataset created and saved as cricket_data.json


# Cypher Queries to Populate the Database

In [5]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 4.6 MB/s eta 0:00:00


In [7]:
from neo4j import GraphDatabase
import json

# Load data
with open("cricket_data.json") as f:
    data = json.load(f)

uri = "neo4j+s://47eebc2d.databases.neo4j.io"  # Replace with your Neo4j Aura Bolt URI
username = "neo4j"  # Replace with your Neo4j username
password = "AjTYFQZ9YqM8y6DFtgDKVNbEo3TRsUkk5a51Cr5o5QY"  # Replace with your Neo4j password

# Connect to Neo4j
driver = GraphDatabase.driver(uri, auth=(username, password))

def populate_db(tx, teams, players, matches):
    # Create teams
    for team in teams:
        tx.run("CREATE (:Team {name: $name, country: $country})", **team)

    # Create players
    for player in players:
        tx.run(
            "CREATE (:Player {name: $name, age: $age, role: $role})",
            name=player["name"], age=player["age"], role=player["role"]
        )
        tx.run(
            "MATCH (p:Player {name: $name}), (t:Team {name: $team}) "
            "CREATE (p)-[:PLAYS_FOR]->(t)", name=player["name"], team=player["team"]
        )

    # Create matches
    for match in matches:
        tx.run(
            "CREATE (:Match {match_id: $match_id, date: $date, location: $location})",
            **match
        )
        tx.run(
            """
            MATCH (t1:Team {name: $team1}), (t2:Team {name: $team2}), (m:Match {match_id: $match_id})
            CREATE (t1)-[:WON]->(m), (t2)-[:LOST]->(m)
            """,
            team1=match["team1"], team2=match["team2"], match_id=match["match_id"]
        )

with driver.session() as session:
    session.execute_write(populate_db, data["teams"], data["players"], data["matches"])

print("Database populated successfully!")


Database populated successfully!


# Players in a Specific Team :



```
MATCH (p:Player)-[:PLAYS_FOR]->(t:Team {name: 'Team A'})
RETURN p.name, p.role
```



In [10]:
def query_players_in_team(tx, team_name):
    result = tx.run(
        "MATCH (p:Player)-[:PLAYS_FOR]->(t:Team {name: $team}) RETURN p.name, p.role",
        team=team_name
    )
    return [record["p.name"] for record in result]

with driver.session() as session:
    players = session.read_transaction(query_players_in_team, "Team B")
    print(players)


<ipython-input-10-a247d20c1343>:9: DeprecationWarning: read_transaction has been renamed to execute_read
  players = session.read_transaction(query_players_in_team, "Team B")


['David Wood', 'Christopher Cruz']


# Matches Played by a Team :

```
MATCH (t:Team {name: 'Team B'})-[:WON|LOST]->(m:Match)
RETURN m.date, m.location
```




In [11]:
def matches_by_team(tx, team_name):
    query = """
    MATCH (t:Team {name: $team_name})-[:WON|LOST]->(m:Match)
    RETURN m.match_id AS MatchID, m.date AS Date, m.location AS Location
    """
    result = tx.run(query, team_name=team_name)
    return [{"MatchID": record["MatchID"], "Date": record["Date"], "Location": record["Location"]} for record in result]

with driver.session() as session:
    team_name = "Team B"
    matches = session.read_transaction(matches_by_team, team_name)
    print(matches)


<ipython-input-11-7bb0659b91ca>:11: DeprecationWarning: read_transaction has been renamed to execute_read
  matches = session.read_transaction(matches_by_team, team_name)


[{'MatchID': 0, 'Date': '2023-07-06', 'Location': 'West Brett'}, {'MatchID': 7, 'Date': '2023-06-04', 'Location': 'South Kathryn'}]


In [12]:
def teams_won_at_location(tx, location):
    query = """
    MATCH (t:Team)-[:WON]->(m:Match {location: $location})
    RETURN t.name AS Team, m.date AS Date
    """
    result = tx.run(query, location=location)
    return [{"Team": record["Team"], "Date": record["Date"]} for record in result]

with driver.session() as session:
    location = "Mumbai"
    teams = session.read_transaction(teams_won_at_location, location)
    print(teams)


<ipython-input-12-10d4a83f1db1>:11: DeprecationWarning: read_transaction has been renamed to execute_read
  teams = session.read_transaction(teams_won_at_location, location)


[{'Team': 'India', 'Date': '2023-10-14'}]


In [13]:
def players_dismissed_by(tx, bowler_name):
    query = """
    MATCH (b:Player {name: $bowler_name})-[:DISMISSED_BY]->(p:Player)
    RETURN p.name AS Batsman
    """
    result = tx.run(query, bowler_name=bowler_name)
    return [record["Batsman"] for record in result]

with driver.session() as session:
    bowler_name = "Jasprit Bumrah"
    dismissed_players = session.read_transaction(players_dismissed_by, bowler_name)
    print(dismissed_players)


<ipython-input-13-8eaa564013be>:11: DeprecationWarning: read_transaction has been renamed to execute_read
  dismissed_players = session.read_transaction(players_dismissed_by, bowler_name)


[]
